### Enter State Farm

In [1]:
import theano

In [2]:
import os, sys
sys.path.insert(1, os.path.join(os.getcwd(), 'utils'))

In [3]:
%matplotlib inline
from __future__ import print_function, division
# path = "data/sample/"
path = "data/statefarm/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [4]:
batch_size = 64

### Create Sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [5]:
%cd data/statefarm
%cd train

/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/train


In [12]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [14]:
for d in glob('c?'):
    os.mkdir('../sample/train/' + d)
    os.mkdir('../sample/valid/' + d)

In [6]:
from shutil import copyfile

In [17]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [23]:
% cd ../../..
%mkdir data/statefarm/results
%mkdir data/statefarm/sample/test

/Users/WayNoxchi/Deshar/Kaukasos/FAI


#### Validation Set (Sample)

How I'll do it: create a full val set in the full valid folder, then copy over the same percentage as train to the sample/valid folder.

Acutally: wouldn't it be better if I used the full validation set for more accurate results? Then again, for processing on my MacBook, it may be good enough to go w/ the 1st method.

##### 1/3: function definitions for moving stuff & aming dirs:

In [7]:
# run once, make sure you're in datadir first
# path = os.getcwd()
# os.mkdir(path + '/valid')
# for i in xrange(10): os.mkdir(path + '/valid' + '/c' + str(i))

def reset_valid(verbose=1, valid_path='', TRAIN_DIR=''):
    """Moves all images in validation set back to 
    their respective classes in the training set."""
    counter = 0
    if not valid_path: valid_path = os.getcwd() + '/valid/'
    if not TRAIN_DIR:  TRAIN_DIR  = os.getcwd() + '/train'
    %cd $valid_path
    for i in xrange(10):
        %cd c"$i"
        g = glob('*.jpg')
        for n in xrange(len(g)):
            os.rename(g[n], TRAIN_DIR + '/c' + str(i) + '/' + g[n])
            counter += 1
        % cd ..
    if verbose: print("Moved {} files.".format(counter))
#         %mv $VALID_DIR/c"$i"/$*.jpg $TRAIN_DIR/c"$i"/$*.jpg

# modified from: http://forums.fast.ai/t/statefarm-kaggle-comp/183/20
def set_valid(number=1, verbose=1, data_path=''):
    """Moves <number> of subjects from training to validation 
    directories. Verbosity: 0: Silent; 1: print no. files moved; 
    2: print each move operation"""
    if not data_path: data_path = os.getcwd() + '/'
    counter = 0
    if number < 0: number = 0
    for n in xrange(number):
        # read CSV file into Pandas DataFrame
        dil = pd.read_csv(data_path + 'driver_imgs_list.csv')
        # group frame by subject in image
        grouped_subjects = dil.groupby('subject')
        # pick <number> subjects at random
        subject = grouped_subjects.groups.keys()[np.random.randint(0, high=len(grouped_subjects.groups))] # <-- groups?
        # get the group assoc w/ subject
        group = grouped_subjects.get_group(subject)
        # loop over gropu & move imgs to validation dir
        for (subject, clssnm, img) in group.values:
            source = '{}train/{}/{}'.format(data_path, clssnm, img)
            target = source.replace('train', 'valid')
            if verbose > 1: print('mv {} {}'.format(source, target))
            os.rename(source, target)
            counter += 1
    if verbose: print ("Files moved: {}".format(counter))

##### 2/3: Making sure we're in the right dir, & moving stuff

In [60]:
%pwd

u'/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid'

In [8]:
%cd ~/Deshar/Kaukasos/FAI

/Users/WayNoxchi/Deshar/Kaukasos/FAI


In [62]:
%cd data/statefarm/
reset_valid()
%cd ..
set_valid(number=3)

/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c0
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c1
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c2
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c3
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c4
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c5
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid/c6
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid
/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid

##### 3/3: copying val set from the full valid folder to sample valid

J.Howard uses a permutation of 1,000 val imgs, so I'll just do that here.

In [67]:
%pwd

u'/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm'

In [69]:
%cd valid
# g = glob('valid/c?/*.jpg') # <-- this doesnt work: why?
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
# for i in range(1000): copyfile(shuf[i], '/sample/' + shuf[i])
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

/Users/WayNoxchi/Deshar/Kaukasos/FAI/data/statefarm/valid


### Create Batches

In [9]:
batches = get_batches(path + 'train', batch_size=batch_size)
val_batches = get_batches(path + 'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, 
    test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


### Basic Models

#### Linear Model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [11]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 224, 224)),
            Flatten(),
            Dense(10, activation='softmax')
        ])

As you can see below, this training is going nowhere...

In [12]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 62s - loss: 13.9601 - acc: 0.0987 - val_loss: 14.0630 - val_acc: 0.1270
Epoch 2/2
1500/1500 [==============================] - 48s - loss: 14.5815 - acc: 0.0953 - val_loss: 14.1033 - val_acc: 0.1250


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________


In [14]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too hgh. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [15]:
np.round(model.predict_generator(batches, batches.n)[:10],2)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [16]:
temp = model.predict_generator(batches, batches.n)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [33]:
# here's a way to take a look at the learning rate
import keras.backend as K
LR = K.eval(model.optimizer.lr)
print(LR)

0.0010000000475


In [35]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Flatten(),
            Dense(10, activation='softmax')
        ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 60s - loss: 2.3591 - acc: 0.2133 - val_loss: 5.1346 - val_acc: 0.1070
Epoch 2/2
1500/1500 [==============================] - 48s - loss: 1.7576 - acc: 0.4280 - val_loss: 3.3038 - val_acc: 0.1950


Great - we found our way out of that hole ... Now we can increase the learning rate and see where we can get to.

In [36]:
model.optimizer.lr=0.001

In [37]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 62s - loss: 1.3820 - acc: 0.6053 - val_loss: 2.5085 - val_acc: 0.2770
Epoch 2/4
1500/1500 [==============================] - 53s - loss: 1.1442 - acc: 0.7033 - val_loss: 2.2965 - val_acc: 0.3320
Epoch 3/4
1500/1500 [==============================] - 51s - loss: 0.9622 - acc: 0.7793 - val_loss: 1.8484 - val_acc: 0.4600
Epoch 4/4
1500/1500 [==============================] - 49s - loss: 0.8183 - acc: 0.8200 - val_loss: 1.5792 - val_acc: 0.5460


We're stabilizing at validation accuracy of 0.39 *(~.50 in my NB)*. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [38]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [40]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]

In [43]:
np.round(val_res,2)

array([[ 1.58,  0.55],
       [ 1.6 ,  0.54],
       [ 1.6 ,  0.54],
       [ 1.57,  0.55],
       [ 1.55,  0.55],
       [ 1.59,  0.54],
       [ 1.57,  0.55],
       [ 1.57,  0.56],
       [ 1.58,  0.55],
       [ 1.57,  0.56]])

Yup, pretty consistent - if we see imporvements of 3% or more, it's probably not random, based on the above samples.

#### L2 Regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (ie: add the sum of squares of the weights to our loss function):

In [44]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Flatten(),
            Dense(10, activation='softmax', W_regularizer=l2(0.01))
        ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 64s - loss: 2.7178 - acc: 0.1647 - val_loss: 5.4327 - val_acc: 0.1270
Epoch 2/2
1500/1500 [==============================] - 51s - loss: 2.0071 - acc: 0.3827 - val_loss: 4.0393 - val_acc: 0.1990


In [45]:
model.optimizer.lr=0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 62s - loss: 1.6172 - acc: 0.5773 - val_loss: 3.1532 - val_acc: 0.2590
Epoch 2/4
1500/1500 [==============================] - 48s - loss: 1.3835 - acc: 0.6647 - val_loss: 2.3724 - val_acc: 0.3750
Epoch 3/4
1500/1500 [==============================] - 49s - loss: 1.1934 - acc: 0.7540 - val_loss: 2.1825 - val_acc: 0.3740
Epoch 4/4
1500/1500 [==============================] - 48s - loss: 1.0514 - acc: 0.8080 - val_loss: 2.0198 - val_acc: 0.4530


Looks like we can get a bit over 50% *(almost, here: 45.3%)* accuracy this way. This'll be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [46]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 224, 224)),
            Flatten(),
            Dense(100, activation='relu'), #¿would λ2 be good here?
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 67s - loss: 1.9732 - acc: 0.3693 - val_loss: 7.3742 - val_acc: 0.1060
Epoch 2/2
1500/1500 [==============================] - 53s - loss: 1.0514 - acc: 0.7080 - val_loss: 3.8568 - val_acc: 0.2200
Epoch 1/5
1500/1500 [==============================] - 66s - loss: 0.5922 - acc: 0.8827 - val_loss: 1.8494 - val_acc: 0.4520
Epoch 2/5
1500/1500 [==============================] - 55s - loss: 0.4276 - acc: 0.9293 - val_loss: 1.3124 - val_acc: 0.5780
Epoch 3/5
1500/1500 [==============================] - 54s - loss: 0.2759 - acc: 0.9700 - val_loss: 0.9763 - val_acc: 0.7210
Epoch 4/5
1500/1500 [==============================] - 56s - loss: 0.1899 - acc: 0.9853 - val_loss: 0.9103 - val_acc: 0.7720
Epoch 5/5
1500/1500 [==============================] - 52s - loss: 0.1516 - acc: 0.9920 - val_loss: 0.9023 - val_acc: 0.7710


*(Odd, I may not have a good validation set if I'm getting such higher valacc numbers...)*

Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single Conv Layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [47]:
def conv1(batches):
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,224,224)),
                Convolution2D(32, 3, 3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3, 3)),
                Convolution2D(64, 3, 3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(200, activation='relu'),
                BatchNormalization(),
                Dense(10, activation='softmax')
            ])
    model.compile(Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches,
                        nb_val_samples=val_batches.nb_sample)
    
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches,
                        nb_val_samples=val_batches.nb_sample)

In [48]:
conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 433s - loss: 1.8576 - acc: 0.4900 - val_loss: 4.1450 - val_acc: 0.1160
Epoch 2/2
1500/1500 [==============================] - 410s - loss: 0.5595 - acc: 0.8873 - val_loss: 2.4079 - val_acc: 0.2540
Epoch 1/4
1500/1500 [==============================] - 436s - loss: 0.1618 - acc: 0.9820 - val_loss: 2.4342 - val_acc: 0.2540
Epoch 2/4
1500/1500 [==============================] - 430s - loss: 0.0788 - acc: 0.9913 - val_loss: 2.8377 - val_acc: 0.1370
Epoch 3/4
1500/1500 [==============================] - 418s - loss: 0.0269 - acc: 1.0000 - val_loss: 2.9813 - val_acc: 0.1300
Epoch 4/4
1500/1500 [==============================] - 426s - loss: 0.0112 - acc: 1.0000 - val_loss: 2.8279 - val_acc: 0.1840


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable results.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data Augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best results we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [49]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [50]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 441s - loss: 2.1688 - acc: 0.3453 - val_loss: 5.9133 - val_acc: 0.1050
Epoch 2/2
1500/1500 [==============================] - 431s - loss: 1.2125 - acc: 0.6373 - val_loss: 2.6613 - val_acc: 0.1700
Epoch 1/4
1500/1500 [==============================] - 417s - loss: 0.7882 - acc: 0.7660 - val_loss: 2.4997 - val_acc: 0.1930
Epoch 2/4
1500/1500 [==============================] - 437s - loss: 0.5987 - acc: 0.8267 - val_loss: 2.6768 - val_acc: 0.1200
Epoch 3/4
1500/1500 [==============================] - 439s - loss: 0.4185 - acc: 0.8820 - val_loss: 3.1566 - val_acc: 0.1800
Epoch 4/4
1500/1500 [==============================] - 453s - loss: 0.3207 - acc: 0.9147 - val_loss: 3.4434 - val_acc: 0.1820


Height shift: move the image up and down - 

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Random shear angles (max in radians) - 

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Rotation: max in degrees - 

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Channel shift: randomly changing the R,B,G colors - 

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path + 'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decision.

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Lucky we tried that - we're starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Amazingly, using nothing but a small sample, a simple (not pre-trianed) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our further experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.